# Chapter 08: LLMs integrating with Forecasting Models

Integrating LLMs with forecasting tools is a powerful strategy for enhancing decision-making across various domains. LLMs excel at interpreting complex datasets and generating actionable insights. When combined with forecasting models, this synergy allows for automatic forecast interpretation and action recommendations. While applications such as demand forecasting and machine degradation predictions are common, our focus will be on earthquake forecasting. We will outline practical steps for integrating LLMs with forecasting models, demonstrating how this combination can transform abstract concepts into actionable and insightful implementations.

In [ ]:
from assets.tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from assets.tools.forecasting import forecast_earthquakes, get_regions
from pydantic import BaseModel, Field
from language_models.models.llm import OpenAILanguageModel
from language_models.agent import Agent, Workflow, WorkflowAgentStep, WorkflowFunctionStep, OutputType, PromptingStrategy
from language_models.tools import Tool
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [ ]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## Use Case: Automating Earthquake Forecast Inquiries

Integrating LLMs with forecasting tools offers a transformative approach to managing inquiries about earthquake forecasts. Currently, handling these inquiries involves significant manual effort, with responses being drafted and sent individually. By automating this process, we can streamline operations and enhance efficiency. To automate responses to earthquake forecast inquiries effectively, we will design a workflow that integrates various steps, starting from the receipt of an email and ending in the generation of an email response, ensuring accurate and timely replies.

**Step 1: Extracting Relevant Information**

To handle the unstructured data in the email body, we first leverage an LLM. The LLM is tasked with extracting key details such as the regions and the forecasting horizons. This step is crucial as it transforms the free-form text into structured data. The LLM's natural language processing capabilities allow it to identify and extract mentions of specific regions and forecasting horizons from the email body.

In [ ]:
get_regions_tool = Tool(
    function=get_regions,
    name="Get Valid Regions",
    description="Use this tool to access the valid regions that can be used for forecasting",
)

In [ ]:
system_prompt = """Use the Get Valid Regions tool to determine the regions the user is interested in.

The email body you receive may contain abbreviations or misspellings.

Make sure to respond with the spelling provided by the Get Valid Regions tool."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=250,
    temperature=0.2,
)

class Forecast(BaseModel):
    horizon: int = Field(description="The number of days to forecast for")
    regions: list[str] = Field(description="The regions to forecast for")

extract_regions = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{email_body}",
    prompt_variables=["email_body"],
    tools=[get_regions_tool],
    output_type=OutputType.OBJECT,
    output_schema=Forecast,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

extract_regions_step = WorkflowAgentStep(name="extract_regions", agent=extract_regions)

**Step 2:**

In [ ]:
class RegionForecast(BaseModel):
    region: str
    forecast: list[dict]

def forecast_earthquakes_for_regions(horizon: int, regions: list[str]) -> list[RegionForecast]:
    forecasts = []
    for region in regions:
        forecast = forecast_earthquakes(region, horizon)
        forecasts.append(RegionForecast(region=region, forecast=forecast))
    return forecasts

forecast_earthquakes_step = WorkflowFunctionStep(name="forecast_earthquakes", inputs=Forecast, function=forecast_earthquakes_for_regions)

**Step 3:**

In [1]:
from functools import reduce

# Sample list of dictionaries with magnitude and depth
data = [
    {'magnitude': 5.1, 'depth': 10.2},
    {'magnitude': 4.8, 'depth': 12.5},
    {'magnitude': 5.5, 'depth': 9.0},
    {'magnitude': 5.0, 'depth': 11.3}
]

def calculate_averages(data):
    # Extract magnitudes and depths
    magnitudes = list(map(lambda x: x['magnitude'], data))
    depths = list(map(lambda x: x['depth'], data))

    # Sum of magnitudes and depths
    total_magnitude = reduce(lambda x, y: x + y, magnitudes)
    total_depth = reduce(lambda x, y: x + y, depths)

    # Number of entries
    count = len(data)

    # Calculate averages
    average_magnitude = total_magnitude / count if count > 0 else 0
    average_depth = total_depth / count if count > 0 else 0

    return {
        'average_magnitude': average_magnitude,
        'average_depth': average_depth
    }

# Calculate and print the average values
averages = calculate_averages(data)
print(averages)


{'average_magnitude': 5.1, 'average_depth': 10.75}


**Step 4:**

**Step 5:**